In [5]:
!sudo apt install build-essential python-dev libxml2 libxml2-dev zlib1g-dev
!sudo pip install cairocffi
!pip install python-igraph
!pip install louvain

import networkx as nx
import igraph as ig
import louvain
from collections import OrderedDict

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
python-dev is already the newest version (2.7.15~rc1-1).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
libxml2 is already the newest version (2.9.4+dfsg1-6.1ubuntu1.2).
libxml2 set to manually installed.
libxml2-dev is already the newest version (2.9.4+dfsg1-6.1ubuntu1.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
     |████████████████████████████████| 71kB 4.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/e7/5d/6f/fc3c2364dfd3c4cfd15d786b156077c52209d9af45496fdf12
Successfully built cairocffi
     |████████████████████████████████| 378kB 4.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/41/d6/02/34eeba

##### Create a total graph and alliance graph
NB: is necessary run this code only one time (you probably have already the net in the dataset folder); i hope wont be necessary come back on this

In [0]:
# create a multigraph for all nets type
messageGraph = nx.MultiDiGraph()
tradeGraph = nx.MultiDiGraph()
attackGraph = nx.MultiDiGraph()

attackSelf = 0
messageSelf = 0

# number of graph (30 in our case)
nGraph = 30

# for every day is needed create total and alliance graph
for i in range(0, nGraph):
    print("Graph number: ", (i + 1))
    
    # for every day is needed reinitialize the "real" graphs (graph with edge label)
    realAttack = nx.MultiDiGraph()
    realMessage = nx.MultiDiGraph()
    realTrade = nx.MultiDiGraph()
    
    attackSelf = 0
    messageSelf = 0
    
    # for every day is needed reinitialize the alliance graph and total graph
    allianceGraph = nx.MultiDiGraph()
    totalGraph = nx.MultiDiGraph()
    
    # read from Dataset/Graphml-File the correct file based on type and day
    attackGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Attacks-Network-Graphml/attacks-timestamped-2009-12-"+str(i+1)+".graphml")
    messageGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-"+str(i+1)+".graphml")
    tradeGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Trades-Network-Graphml/trades-timestamped-2009-12-"+str(i+1)+".graphml")
    
    # copy xxxxGraph in realxxxx adding label to edges; keep attentio about key, it must be unique (we use old key + 
    # letter for define edge tipology) 
    for itr in list(messageGraph.edges(None, True, True)):
    #for itr in messageGraph.edges_iter(None, True, True):
        if int(itr[0]) != int(itr[1]):
            realMessage.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"m", Label= "Message", time = itr[3]['edgetime'])
        else:
            messageSelf += 1
    
    #for itr in tradeGraph.edges_iter(None, True, True):
    for itr in list(tradeGraph.edges(None, True, True)):
        realTrade.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"t", Label = "Trade", time = itr[3]['edgetime'])

    #for itr in attackGraph.edges_iter(None, True, True):
    for itr in list(attackGraph.edges(None, True, True)):
        if int(itr[0]) != int(itr[1]):
            realAttack.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"a", Label= "Attack", time = itr[3]['edgetime'])
        else:
            attackSelf += 1
    
    # compose (a union) in one graph the needed graphs 
    allianceGraph = nx.compose(realMessage, realTrade)
    totalGraph = nx.compose(allianceGraph, realAttack)
    # save total graph and alliance graph in the correct file
    nx.write_graphml(totalGraph, baseURL + "Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-"+str(i+1)+".graphml")
    nx.write_graphml(allianceGraph, baseURL + "Dataset/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-"+str(i+1)+".graphml")

    # print for check, i checked that sum of graphs is the equal to alliance graph and total graph (according to the case)
    # but i think is a better that YOU MAKE THE SAME; ONE MORE CONTROL IS BETTER!
    print("ATTACK GRAPH: ", nx.info(attackGraph))
    print("NO SENSE SELF ATTACK EDGE: ", attackSelf)
    print("MESSAGE GRAPH: ", nx.info(messageGraph))
    print("NO SENSE SELF MESSAGE EDGE: ", messageSelf)
    print("TRADE GRAPH: ", nx.info(tradeGraph))
    print("ALLIANCE GRAPH: ", nx.info(allianceGraph))
    print("TOTAL GRAPH: ", nx.info(totalGraph))
    print("") 

Graph number:  1
ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2807
Number of edges: 32997
Average in degree:  11.7553
Average out degree:  11.7553
NO SENSE SELF ATTACK EDGE:  0
MESSAGE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1963
Number of edges: 16346
Average in degree:   8.3271
Average out degree:   8.3271
NO SENSE SELF MESSAGE EDGE:  515
TRADE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1336
Number of edges: 9390
Average in degree:   7.0284
Average out degree:   7.0284
ALLIANCE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2218
Number of edges: 25221
Average in degree:  11.3711
Average out degree:  11.3711
TOTAL GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 3757
Number of edges: 58218
Average in degree:  15.4959
Average out degree:  15.4959

Graph number:  2
ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2802
Number of edges: 32782
Average in degree:  11.6995
Average out degree:  11.6995
NO SENSE SELF ATTACK EDGE:  0
MESSAGE GRAPH:

ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2082
Number of edges: 21628
Average in degree:  10.3881
Average out degree:  10.3881
NO SENSE SELF ATTACK EDGE:  0
MESSAGE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1833
Number of edges: 14447
Average in degree:   7.8816
Average out degree:   7.8816
NO SENSE SELF MESSAGE EDGE:  402
TRADE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1145
Number of edges: 9310
Average in degree:   8.1310
Average out degree:   8.1310
ALLIANCE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1996
Number of edges: 23355
Average in degree:  11.7009
Average out degree:  11.7009
TOTAL GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2965
Number of edges: 44983
Average in degree:  15.1713
Average out degree:  15.1713

Graph number:  13
ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2066
Number of edges: 20675
Average in degree:  10.0073
Average out degree:  10.0073
NO SENSE SELF ATTACK EDGE:  0
MESSAGE GRAPH:  Name: 
Type: M

ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1732
Number of edges: 16477
Average in degree:   9.5133
Average out degree:   9.5133
NO SENSE SELF ATTACK EDGE:  0
MESSAGE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1632
Number of edges: 13178
Average in degree:   8.0748
Average out degree:   8.0748
NO SENSE SELF MESSAGE EDGE:  289
TRADE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1013
Number of edges: 8062
Average in degree:   7.9585
Average out degree:   7.9585
ALLIANCE GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1764
Number of edges: 20951
Average in degree:  11.8770
Average out degree:  11.8770
TOTAL GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 2610
Number of edges: 37428
Average in degree:  14.3402
Average out degree:  14.3402

Graph number:  24
ATTACK GRAPH:  Name: 
Type: MultiDiGraph
Number of nodes: 1716
Number of edges: 16467
Average in degree:   9.5962
Average out degree:   9.5962
NO SENSE SELF ATTACK EDGE:  0
MESSAGE GRAPH:  Name: 
Type: M

In [0]:
def weightCompute(scoreList):
    if scoreList[0] > 0 and scoreList[1] > 0:
        return 2*sum(scoreList)
    else:
        return sum(scoreList)

Enable Google Drive on Colab

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')
baseURL = '/content/gdrive/My Drive/Data Analytics/';

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


Or use your local path on Jupyter

In [0]:
baseURL = '../'

Code to transform the Multigraph to the plain alliance graph

---

Do not run if you read the alliance graph from the file

In [15]:

nGraph = 30
realAllianceGraph = nx.DiGraph()


checkGraph = ig.Graph()
edgesDict = dict()

tradeCount = 0
messageCount = 0


for i in range(0, nGraph):
    
    allianceGraph = nx.MultiDiGraph()
    realAllianceGraph = nx.DiGraph()
    checkGraph = ig.Graph()
    allianceGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-"+str(i+1)+".graphml")
    checkGraph = checkGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-"+str(i+1)+".graphml")
    

    tradeCount = 0
    messageCount = 0
    sumOfVertex = 0
    nStep = 0
    zeroOutDegreeCont = 0
    edgesDict = dict()

    print("DAY: ", (i+1))
    print("START GRAPH")
    print(nx.info(allianceGraph))
    print("")
    
    #for itr in allianceGraph.edges_iter(None, True, True):
    for itr in list(allianceGraph.edges(None, True, True)):
        print(itr)
        if itr[0] in edgesDict:
            if itr[1] in edgesDict[itr[0]]:
                
                if itr[3]['Label'] == 'Message':
                    edgesDict[itr[0]][itr[1]][0] += 1
                    messageCount += 1
                elif itr[3]['Label'] == 'Trade':
                    edgesDict[itr[0]][itr[1]][1] += 1
                    tradeCount += 1
            else:
                
                if itr[3]['Label'] == 'Message':
                    edgesDict[itr[0]][itr[1]] = [1, 0]
                    messageCount += 1
                elif itr[3]['Label'] == 'Trade':
                    edgesDict[itr[0]][itr[1]] = [0, 1]
                    tradeCount += 1
        else:
            edgesDict[itr[0]] = {}
            
            if itr[3]['Label'] == 'Message':
                edgesDict[itr[0]][itr[1]] = [1, 0]
                messageCount += 1
            elif itr[3]['Label'] == 'Trade':
                edgesDict[itr[0]][itr[1]] = [0, 1]
                tradeCount += 1

    for fromVertex in edgesDict:
        for toVertex in edgesDict[fromVertex]:
            nStep += 1
            realAllianceGraph.add_edge(int(fromVertex), int(toVertex), Weight = weightCompute(edgesDict[fromVertex][toVertex]))
            '''print("FROM VERTEX: ", fromVertex)
            print("VALUE: ", edgesDict[fromVertex])
            print("TO VERTEX: ", toVertex)
            print("VALUE: ", edgesDict[fromVertex][toVertex])
            print("SUM: ", sum(edgesDict[fromVertex][toVertex]))'''
            sumOfVertex += sum(edgesDict[fromVertex][toVertex])

        
    print("FROM VERTICES NUMBER IN DICTIONARY: ", len(edgesDict))
    print("START GRAPH ALL VERTICES: ", len(allianceGraph))

    for vertex in checkGraph.vs:
        if vertex.outdegree() == 0:
            zeroOutDegreeCont += 1
    print("VERTICES WITH OUTDEGREE 0:", zeroOutDegreeCont)
    print("")

    print("END GRAPH")
    print(nx.info(realAllianceGraph))
    print("")
    # for itr in realAllianceGraph.edges_iter(None, True, True):
        # print(itr)
    print("SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER): ", sumOfVertex)
    print("NUMBER OF STEP (END GRAPH EDGES NUMBER): ", nStep)
    print("NUMBER OF TRADE EDGES MET: ", tradeCount)
    print("NUMBER OF MESSAGE EDGES MET: ", messageCount)
    nx.write_graphml(realAllianceGraph, baseURL + "Dataset/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-"+str(i+1)+".graphml")
    print('----------------------------------------------------------------------')

    

DAY:  1
START GRAPH
Name: 
Type: DiGraph
Number of nodes: 2218
Number of edges: 11077
Average in degree:   4.9941
Average out degree:   4.9941

('8603', '282', {'Weight': 1})


IndexError: ignored

##### Creation of our structure for maintain all nets
Create a dictionary of lists, the key rappresents the day (start 0 end 29), the list is a list of nets about the specific day with this order: (attackGraph, messageGraph, TradeGraph, allianceGraph, totalGraph)

In [16]:
# our structure 
allNetwork = dict()

nGraph = 30

# for structure graphs inserting
attackGraph = ig.Graph()
messageGraph = ig.Graph()
tradeGraph = ig.Graph()
totalGraph = ig.Graph()
allianceGraph = ig.Graph()

for i in range(0, nGraph):
    print("Day: ", (i + 1))
    
    # read all needed graphs
    attackGraph = attackGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Attacks-Network-Graphml/attacks-timestamped-2009-12-"+str(i+1)+".graphml")
    messageGraph = messageGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-"+str(i+1)+".graphml")
    tradeGraph = tradeGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Trades-Network-Graphml/trades-timestamped-2009-12-"+str(i+1)+".graphml")
    allianceGraph = allianceGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-"+str(i+1)+".graphml")
    totalGraph = totalGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-"+str(i+1)+".graphml")
    
    # put as a list in a dictionary 
    allNetwork[i] = (attackGraph, messageGraph, tradeGraph, allianceGraph, totalGraph) 

Day:  1
Day:  2
Day:  3
Day:  4
Day:  5
Day:  6
Day:  7
Day:  8
Day:  9
Day:  10
Day:  11
Day:  12
Day:  13
Day:  14
Day:  15
Day:  16
Day:  17
Day:  18
Day:  19
Day:  20
Day:  21
Day:  22
Day:  23
Day:  24
Day:  25
Day:  26
Day:  27
Day:  28
Day:  29
Day:  30


### This below cell will be use for any kind of check about nets, variables, indexes; for this reason no one comments are required

In [17]:
for i in range(0, len(allNetwork)):
    for vertex in allNetwork[i][4].vs:
        if vertex['id'] == '2789':
            print("{:>20}{:>2}".format("Present in the day:", str(i + 1)))
print("")     
for i in range(0, len(allNetwork)):
    print("DAY: "+str(i+1))
    for edge in allNetwork[i][4].es:
        if allNetwork[i][4].vs[edge.tuple[0]]['id'] == '2789':
            print(edge)
print("")
for i in range(0, len(allNetwork)):
    print("DAY: "+str(i+1))
    for edge in allNetwork[i][4].es:
        if allNetwork[i][4].vs[edge.tuple[1]]['id'] == '2789':
            print(edge)
        

# 11850

 Present in the day: 1
 Present in the day: 2
 Present in the day: 3
 Present in the day: 4
 Present in the day: 5
 Present in the day: 6
 Present in the day: 7
 Present in the day: 8
 Present in the day: 9
 Present in the day:10
 Present in the day:11
 Present in the day:12
 Present in the day:13
 Present in the day:14
 Present in the day:15
 Present in the day:16
 Present in the day:17
 Present in the day:18
 Present in the day:19
 Present in the day:20
 Present in the day:21
 Present in the day:22
 Present in the day:24
 Present in the day:25
 Present in the day:26
 Present in the day:28

DAY: 1
igraph.Edge(<igraph.Graph object at 0x7ff9a398f138>, 54281, {'time': 1259690340.0, 'Label': 'Trade', 'id': '0t'})
igraph.Edge(<igraph.Graph object at 0x7ff9a398f138>, 54282, {'time': 1259701108.0, 'Label': 'Trade', 'id': '1t'})
igraph.Edge(<igraph.Graph object at 0x7ff9a398f138>, 54283, {'time': 1259684008.0, 'Label': 'Trade', 'id': '2t'})
igraph.Edge(<igraph.Graph object at 0x7ff9a398f138>,

# START ANALYSIS !!!!

### Users number trend in the 30 days

In [0]:
def userNumberTrend(allNetwork):
    nUsers = [0]*len(allNetwork)
    for i in range(0, len(allNetwork)):
        nUsers[i] = allNetwork[i][4].vcount()

    print("Users in the 30 days: ", nUsers)
    print("")
    print("{:>53}{:>7}.".format("Average users number in the 30 days is: ", sum(nUsers)/len(nUsers)))
    print("{:>53}{:>7}.".format("The maximum number of users was recorded on the day: ", nUsers.index(max(nUsers))+1))
    print("")

### Edges number trend in the 30 days

In [0]:
def edgeNumberTrend(allNetwork):  
    nAttack = [0]*len(allNetwork)
    nTrade = [0]*len(allNetwork)
    nMessage = [0]*len(allNetwork)

    for i in range(0, nGraph):
        for edge in allNetwork[i][4].es:
            if edge['Label'] == "Attack":
                nAttack[i] += 1
            elif edge['Label'] == "Trade":
                nTrade[i] += 1
            elif edge['Label'] == "Message":
                nMessage[i] += 1
            else:
                print("This is a problem...")

    print("{:>33}".format("Number of attacks in the 30 days:"), end = " ")
    print(nAttack)
    print("")
    print("{:>32}".format("Number of trades in the 30 days:"), end = " ")
    print(nTrade)
    print("")
    print("{:>34}".format("Number of messages in the 30 days:"), end = " ")
    print(nMessage)
    print("")
    print("{:>40}{:>9}".format("Average number of attacks in 30 days:", sum(nAttack)/len(nAttack)))
    print("{:>40}{:>9}".format("Average number of trades a in 30 days:", round(sum(nTrade)/len(nTrade), 1)))
    print("{:>40}{:>9}".format("Average number of messages a in 30 days:", round(sum(nMessage)/len(nMessage), 1)))  
    print("")
    print("{:>40}{:>9}".format("Day with the highest number of attacks:", nAttack.index(max(nAttack))+1))
    print("{:>40}{:>9}".format("Day with the highest number of trades:", nTrade.index(max(nTrade))+1))
    print("{:>40}{:>9}".format("Day with the highest number of messages:", nMessage.index(max(nMessage))+1)) 
    print("")

### Trickster detection tool 

In [0]:
def printRealSuspected(allSuspects, nDays):
    
    beneficiaries = dict()
    suspectsSummary = dict()
    finalSuspects = dict()
    
    for day in allSuspects:
        for suspect in allSuspects[day]:
            # print("Suspected: ", suspect)
            if str(suspect) in suspectsSummary:
                suspectsSummary[suspect][0] += 1
            else:
                suspectsSummary[suspect] = list([1, dict(), 0, 0])
                suspectsSummary[suspect][0] = 1
                # print(suspectsSummary[suspect][0])
            # print("TIME: ", suspectsSummary[suspect])
            
            for beneficiary in allSuspects[day][suspect]:
                # print("Beneficiary is: ", beneficiary)
                # print("Benefit is: ", allSuspects[day][suspect][beneficiary])
                # print(str(beneficiary) in suspectsSummary[suspect][1])
                if str(beneficiary) in suspectsSummary[suspect][1]:
                    suspectsSummary[suspect][1][beneficiary][0] += 1
                    suspectsSummary[suspect][1][beneficiary][1] += allSuspects[day][suspect][beneficiary]
                else:
                    suspectsSummary[suspect][1][beneficiary] = [1, allSuspects[day][suspect][beneficiary]]

                    
            # print(suspectsSummary)

    for day in allSuspects:
        for suspect in allSuspects[day]:
            for beneficiary in allSuspects[day][suspect]:
                if beneficiary in suspectsSummary:
                    # print("HERE MAN")
                    suspectsSummary[beneficiary][2] += 1
                    suspectsSummary[beneficiary][3] += allSuspects[day][suspect][beneficiary]
                    # print(suspectsSummary[beneficiary])
                    
            # print(suspectsSummary)
            # print("")
                
    # suspectsSummary = sorted(suspectsSummary.items(), key=lambda e: e[1][2])
    suspectsSummary = OrderedDict(sorted(suspectsSummary.items(), key=lambda t: t[1][0], reverse=True))
    
    print("Complete: ", len(suspectsSummary))
    for suspect in suspectsSummary:
        print(suspect, suspectsSummary[suspect])
        
    for suspect in suspectsSummary:
        if suspectsSummary[suspect][0] > 1 and suspectsSummary[suspect][0] != suspectsSummary[suspect][2]:
            finalSuspects[suspect] = suspectsSummary[suspect]
            
    # print("With elimination: ", len(finalSuspects))
    # for suspect in finalSuspects:
        # print(suspect, finalSuspects[suspect])
    
    # print("")
    # print(finalSuspects)
    
    return finalSuspects
                    
def tricksterDetectionTool(totalNet):
    zeroIndegreeVertex = list()

    for vertex in totalNet.vs:
        if vertex.indegree() == 0:
            zeroIndegreeVertex.append(vertex)
    
    '''for vertex in zeroIndegreeVertex:
        print(vertex)
        print(vertex.indegree())
        print(vertex.outdegree())'''
    #print(zeroIndegreeVertex)
    
    suspecteds = dict()
    beneficiary = dict()
    key = ""
    
    for vertex in zeroIndegreeVertex:
        nEdges = 0
        nTrades = 0
        key = ""
        beneficiary = dict()
        for edge in totalNet.es:
            if totalNet.vs[edge.tuple[0]]['id'] == vertex['id']:
                nEdges += 1
                if edge['Label'] == 'Trade':
                    # print(edge)
                    # print(totalNet.vs[edge.tuple[0]]['id'])
                    # print(vertex['id'])
                    if totalNet.vs[edge.tuple[1]]['id'] in beneficiary:
                        beneficiary[totalNet.vs[edge.tuple[1]]['id']] += 1
                    else:
                        beneficiary[totalNet.vs[edge.tuple[1]]['id']] = 1
                    nTrades += 1
        if nTrades/nEdges >= 0.95:
            # print(nEdges)
            # print(nTrades)
            key = str(vertex['id'])
            # print(key)
            # print(beneficiary)
            suspecteds[key] = beneficiary
    
    for key in suspecteds:
        print(key, suspecteds[key])
    return suspecteds

def edgesCount(finalSuspects, allNet):
    edgesInCount = dict()
    edgesOutCount = dict()
    edgesOutPlayer = dict()
    
    activityDay = list()
    
    inCount = 0
    outCount = 0
    tradeOutCount = 0
    tradeInCount = 0
    
    
    for suspect in finalSuspects:
        
        activityDay.clear()
        edgesInCount.clear()
        edgesOutCount.clear()
        edgesOutPlayer.clear()
        
        inCount = 0
        outCount = 0
        tradeOutCount = 0
        tradeInCount = 0
        
        for day in allNet:
            for vertex in allNet[day][4].vs:
                if str(vertex['id']) == suspect:
                    activityDay.append(str(day+1))
            


            for edge in allNet[day][4].es:
                if allNetwork[day][4].vs[edge.tuple[0]]['id'] == suspect:
                    if edge['Label'] in edgesOutCount:
                        edgesOutCount[edge['Label']] += 1
                    else:
                        edgesOutCount[edge['Label']] = 1
                    if edge['Label'] == 'Trade':
                        if allNetwork[day][4].vs[edge.tuple[1]]['id'] in edgesOutPlayer:
                            edgesOutPlayer[allNetwork[day][4].vs[edge.tuple[1]]['id']] += 1
                        else:
                            edgesOutPlayer[allNetwork[day][4].vs[edge.tuple[1]]['id']] = 1
                if allNet[day][4].vs[edge.tuple[1]]['id'] == suspect:
                    if edge['Label'] in edgesInCount:
                        edgesInCount[edge['Label']] += 1
                    else:
                        edgesInCount[edge['Label']] = 1
        
        print("SUSPECT: ", suspect)
        print("EDGES IN: ", edgesInCount)
        print("EDGES OUT: ", edgesOutCount)
        print("PLAYER SERVED", edgesOutPlayer)
        print("ACTIVITY DAY: ", activityDay)
        
    
    

## Communities Detection

### TODO
TODO

In [0]:
def startCommunitiesDetection(allianceNet):
    getLouvainCommunities(allianceNet)
    # getEigenvectorCommunities(allianceNet)
    # getOptiModulaCommunities(allianceNet)
    

def getLouvainCommunities(allianceNet):
    partition = louvain.find_partition(allianceNet, louvain.ModularityVertexPartition)
    print("PARTITION: ", partition)
    print("PARTITION MODULARITY: ", partition.modularity)
    print("PARTITION MEMBER: ", partition.membership)
    print("SUMMARY: ", partition.summary())
    print("SIZE: ", partition.sizes())

def getEigenvectorCommunities(allianceNet):
    partition_e = allianceNet.community_leading_eigenvector()
    print("PARTITION: ", partition_e)
    print("PARTITION MODULARITY: ", partition_e.modularity)
    print("PARTITION MEMBER: ", partition_e.membership)
    print("SUMMARY: ", partition_e.summary())
    print("SIZE: ", partition_e.sizes())
    

def getOptiModulaCommunities(allianceNet):
    partition_o = allianceNet.community_optimal_modularity()
    print("PARTITION: ", partition_o)
    print("PARTITION MODULARITY: ", partition_o.modularity)
    print("PARTITION MEMBER: ", partition_o.membership)
    print("SUMMARY: ", partition_o.summary())
    print("SIZE: ", partition_o.sizes())

### TODO
TODO

In [0]:
def VisualizeNetwork(allianceNet):
    print("USING Plotly Python Open Source Graphing Library")

## Main
From cell below we let's start all our analysis

In [26]:
#      TRY, IF YOU WANT!    #
 ###########################
#userNumberTrend(allNetwork)#
#edgeNumberTrend(allNetwork)#
 ########################### 


dailySuspicions = dict()
dailySuspicions.clear()
for i in range(0, len(allNetwork)):
    print("DAY: ", str(i+1))
    dailySuspicions[i] = tricksterDetectionTool(allNetwork[i][4])

    # print(dailySuspicions)
    print("")

'''for day in dailySuspicions:
    print("DAY: ", int(day+1))
    for suspect in dailySuspicions[day]:
        print(suspect, dailySuspicions[day][suspect])
    print("")'''

edgesCount(printRealSuspected(dailySuspicions, len(dailySuspicions)), allNetwork)

# startCommunitiesDetection(allNetwork[0][3])



DAY:  1
2747 {'2734': 3, '2247': 32}
7118 {'6743': 7}
8972 {'3893': 1}
1816 {'3413': 1}
8041 {'5332': 2}
7857 {'3778': 3}
9192 {'3778': 2}
11400 {'12027': 2}
7855 {'3778': 2}
2789 {'1451': 6}
5116 {'5794': 2}
6553 {'10843': 3}
11407 {'11388': 5}
3801 {'5019': 4}
7741 {'2119': 4}
11745 {'10251': 1}
5817 {'9522': 1}
10242 {'4766': 1}
11961 {'12073': 1}
10279 {'3282': 1}
10603 {'12091': 1, '10649': 1, '12088': 1, '12089': 1}

DAY:  2
10008 {'1457': 1, '1339': 1}
8041 {'5332': 3}
10000 {'107': 6}
2789 {'1451': 10}
11407 {'11388': 4, '11106': 1}
803 {'5341': 1}
10358 {'3282': 1}
11661 {'11636': 2, '11381': 4}
1306 {'1480': 1}
743 {'1669': 1}
5092 {'9417': 1}
5817 {'9522': 1}
11745 {'10251': 1}
10651 {'10649': 2}
7551 {'1192': 2}

DAY:  3
3545 {'2553': 1}
7393 {'3895': 1}
8041 {'5332': 4}
1420 {'1451': 9}
11407 {'11106': 2, '11388': 3}
2789 {'1451': 9}
11661 {'11636': 1, '11381': 1}
11647 {'11636': 1}
12044 {'4611': 7}
7118 {'6743': 7}
4034 {'5184': 1}
10000 {'107': 3}
10795 {'4926': 3}
2747

KeyboardInterrupt: ignored

# I prefer keep a bin cell rather than cry because i need already written code but deleted....
Of course in the final release this will be delete

In [0]:
# for libraries install
'''!sudo apt install build-essential python-dev libxml2 libxml2-dev zlib1g-dev
!sudo pip install cairocffi
!pip install python-igraph
!pip install louvain'''

attackGraph = ig.Graph()
messageGraph = ig.Graph()
tradeGraph = ig.Graph()

totalPresent = False
alliancePresent = False

situa = 0

graphs = dict()
dayGraphs = list()

nGraph = 30

for i in range(0, 1):
    totalGraph = ig.Graph()
    
    messageGraph = messageGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-"+str(i+1)+".graphml")
    for messageVertex in messageGraph.vs:
        if messageVertex.degree() != 0:
            allianceGraph.add_vertices(messageVertex['id'])
        totalPresent = False
        for totalVertex in totalGraph.vs:
            if messageVertex['id'] == totalVertex['name']:
                totalPresent = True
                break
        if not totalPresent:
            if messageVertex.degree() != 0:
                totalGraph.add_vertices(messageVertex['id'])
        totalPresent = False
    
    print("MESSAGE EDGE")        
    for edge in messageGraph.es:
        # edge['edgelabel'] = "Message"
        # print(situa, end = "")
        # print(", ", end = "")
        # situa += 1
        totalGraph.add_edge(messageGraph.vs[edge.tuple[0]], messageGraph.vs[edge.tuple[1]], edgelabel='Message')
        # allianceGraph.add_edge(edge.tuple[0], edge.tuple[1], edgelabel='Message')
    # situa = 0
    # print("")
        

    

    print("MESSAGE GRAPH:", messageGraph.summary())
    print("TOTAL GRAPH:", totalGraph.summary())
    
    print("")
    
    for vertex in totalGraph.vs:
    if vertex['name'] == '9152':
        print(vertex.outdegree())
attackGraph = ig.Graph()
messageGraph = ig.Graph()
tradeGraph = ig.Graph()

totalPresent = False
alliancePresent = False

situa = 0

graphs = dict()
dayGraphs = list()

nGraph = 30

for i in range(0, nGraph):
    totalGraph = ig.Graph()
    allianceGraph = ig.Graph()
    print("Graph number: ", (i + 1))
    
    attackGraph = attackGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Attacks-Network-Graphml/attacks-timestamped-2009-12-"+str(i+1)+".graphml")
    for vertex in attackGraph.vs:
        totalGraph.add_vertices(vertex['id'])
        # print(vertex['id'] +", ", end="")
    # print("")
    print("ATTACK EDGE")
    for edge in attackGraph.es:
        # edge['edgelabel'] = "Attack"
        # print(situa, end = "")
        # print(", ", end = "")
        # situa += 1
        totalGraph.add_edge(attackGraph.vs[edge.tuple[0]], attackGraph.vs[edge.tuple[1]], edgelabel='Attack') 
    situa = 0
    # print("")
    
    messageGraph = messageGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-"+str(i+1)+".graphml")
    for messageVertex in messageGraph.vs:
        allianceGraph.add_vertices(messageVertex['id'])
        totalPresent = False
        for totalVertex in totalGraph.vs:
            if messageVertex['id'] == totalVertex['name']:
                totalPresent = True
                break
        if not totalPresent:
            totalGraph.add_vertices(messageVertex['id'])
        totalPresent = False
    
    print("MESSAGE EDGE")        
    for edge in messageGraph.es:
        # edge['edgelabel'] = "Message"
        # print(situa, end = "")
        # print(", ", end = "")
        # situa += 1
        totalGraph.add_edge(messageGraph.vs[edge.tuple[0]], messageGraph.vs[edge.tuple[1]], edgelabel='Message')
        allianceGraph.add_edge(messageGraph.vs[edge.tuple[0]], messageGraph.vs[edge.tuple[1]], edgelabel='Message')
    # situa = 0
    # print("")
        
    tradeGraph = tradeGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Trades-Network-Graphml/trades-timestamped-2009-12-"+str(i+1)+".graphml")
    for tradeVertex in tradeGraph.vs:
        totalPresent = False
        for totalVertex in totalGraph.vs:
            if tradeVertex['id'] == totalVertex['name']:
                totalPresent = True
                break
        if not totalPresent:
            totalGraph.add_vertices(tradeVertex['id'])
        
    for tradeVertex in tradeGraph.vs:
        alliancePresent = False
        for allianceVertex in allianceGraph.vs:
            if tradeVertex['id'] == allianceVertex['name']:
                alliancePresent = True
                break
        if not alliancePresent:
            allianceGraph.add_vertices(tradeVertex['id'])
        
    
    print("TRADE EDGE")
    for edge in tradeGraph.es:
        # edge['edgelabel'] = "Trade"
        # print(situa, end = "")
        # print(", ", end = "")
        # situa += 1
        totalGraph.add_edge(tradeGraph.vs[edge.tuple[0]], tradeGraph.vs[edge.tuple[1]], edgelabel='Trade')
        allianceGraph.add_edge(tradeGraph.vs[edge.tuple[0]], tradeGraph.vs[edge.tuple[1]], edgelabel='Trade')
    situa = 0
    # allianceGraph = messageGraph.union(tradeGraph)
    # totalGraph = allianceGraph.union(attackGraph)
    # print("")
    
    print("ATTACK GRAPH:", attackGraph.summary())
    print("MESSAGE GRAPH:", messageGraph.summary())
    print("TRADE GRAPH:", tradeGraph.summary())
    print("ALLIANCE GRAPH:", allianceGraph.summary())
    print("TOTAL GRAPH:", totalGraph.summary())
    totalGraph.save(baseURL + "Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-"+str(i+1)+".graphml")
    allianceGraph.save(baseURL + "Dataset/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-"+str(i+1)+".graphml")
    print("")
    






'''G = ig.Graph()
G = G.Read_GraphML(baseURL + "Dataset/Graphml-File/Trades-Network-Graphml/trades-timestamped-2009-12-1.graphml") 
G += G.Read_GraphML(baseURL + "Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-1.graphml") 
G += G.Read_GraphML(baseURL + "Dataset/Graphml-File/Attacks-Network-Graphml/attacks-timestamped-2009-12-1.graphml") 
print(G)'''


# print(G.vs.attribute_names())
# print(G.es.attribute_names())
# print(G.degree())
# print(G.indegree())
# print(G.outdegree())

# totalGraph = totalGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-1.graphml")
allianceGraph = ig.Graph()
allianceGraph = allianceGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-1.graphml")

totalGraph = ig.Graph()
totalGraph = totalGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-1.graphml")

attackGraph = ig.Graph()
attackGraph = attackGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Attacks-Network-Graphml/attacks-timestamped-2009-12-1.graphml")

messageGraph = ig.Graph()
messageGraph = messageGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-1.graphml") 

tradeGraph = ig.Graph()
tradeGraph = tradeGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Trades-Network-Graphml/trades-timestamped-2009-12-1.graphml")

# print(totalGraph.degree())
# print(messageGraph.degree())
# print(attackGraph.degree())
# print(tradeGraph.degree())
print("AllianceGraph vertex with 0 degree")
for vertex in allianceGraph.vs:
    if vertex.degree() == 0:
        print(vertex['id'] +", ", end = "")
print("")
print("")
print("TotalGraph vertex with 0 degree")
for vertex in totalGraph.vs:
    if vertex.degree() == 0:
        print(vertex['id']+", ", end="")
print("")
print("")

for vertex in attackGraph.vs:
    if vertex.degree() == 0:
        print(vertex['id']+", ", end="")

print("")
print("")
ok = False
print("Attack Graph Check in Total Graph")
for vertex1 in attackGraph.vs:
    ok = False
    if vertex1['id'] == 'n2807':
        print("i am here 1")
    for vertex2 in totalGraph.vs:
        if vertex1['id'] == vertex2['name']:
            ok = True
            break
    if not ok:
        print("error")

print("")
print("")
ok = False
print("Message Graph Check in Total Graph")
for vertex1 in messageGraph.vs:
    ok = False
    if vertex1['id'] == 'n2807':
        print("i am here 2")
    for vertex2 in totalGraph.vs:
        if vertex1['id'] == vertex2['name']:
            ok = True
            break
    if not ok:
        print("error")
        
print("")
print("")
ok = False
print("Trade Graph Check in Total Graph")
for vertex1 in tradeGraph.vs:
    if vertex1['id'] == 'n2807':
        print("i am here 3")
    ok = False
    for vertex2 in totalGraph.vs:
        if vertex1['id'] == vertex2['name']:
            ok = True
            break
    if not ok:
        print("error")

print("")
print("")
ok = False
print("Message Graph Check in Alliance Graph")
for vertex1 in messageGraph.vs:
    ok = False
    for vertex2 in allianceGraph.vs:
        if vertex1['id'] == vertex2['name']:
            ok = True
            break
    if not ok:
        print("error")
        
print("")
print("")
ok = False
print("Trade Graph Check in Alliance Graph")
for vertex1 in tradeGraph.vs:
    ok = False
    for vertex2 in allianceGraph.vs:
        if vertex1['id'] == vertex2['name']:
            ok = True
            break
    if not ok:
        print("error")
    
print(allianceGraph.summary())
print(totalGraph.summary())

for vertex in totalGraph.vs:
    if vertex['id'] == 'n2807':
        print(vertex)
print("")
print("show in the graph")
for vertex in totalGraph.vs:
    if vertex['name'] == '9152':
        print(vertex)
    
print("In Total Graph")
for edge in totalGraph.es:
    if edge.tuple[0] == '9152' or edge.tuple[1] == '9152':
        print(edge)

print("In Message Graph")
for edge in messageGraph.es:
    if edge.tuple[0] == '9152':
        print(edge)
    if edge.tuple[1] == '9152':
        print(edge)
print("")
for vertex in attackGraph.vs:
    if vertex['id'] == '9152':
        print("attack")
        print(vertex)

for vertex in tradeGraph.vs:
    if vertex['id'] == '9152':
        print("trade")
        print(vertex)


for vertex in messageGraph.vs:
    if vertex['id'] == '9152':
        print("message")
        print(vertex)

        
list(attackGraph.vs)

import networkx as nx

messageGraph = nx.MultiDiGraph()
tradeGraph = nx.MultiDiGraph()
attackGraph = nx.MultiDiGraph()
allianceGraph = nx.MultiDiGraph()
totalGraph = nx.MultiDiGraph()

realMessage = nx.MultiDiGraph()
realTrade = nx.MultiDiGraph()
realAttack = nx.MultiDiGraph()

messageGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-1.graphml")
tradeGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Trades-Network-Graphml/trades-timestamped-2009-12-1.graphml")
attackGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Attacks-Network-Graphml/attacks-timestamped-2009-12-1.graphml")
# nx.info(messageGraph)

# print(messageGraph.nodes()[0])

# for edge in messageGraph.edges():   
#    messageGraph[edge[0]][edge[1]].update(label= "Message")

# nx.set_edge_attributes(messageGraph, "Label", "Message")
# nx.set_edge_attributes(tradeGraph, "Label", "Trade")
# nx.set_edge_attributes(attackGraph, "Label", "Attack")

# allianceGraph = nx.disjoint_union(messageGraph, tradeGraph)

# nx.set_edge_attributes(G, bb, 'betweenness')

# for itr in allianceGraph.edges_iter(None, True, True):
#        print(itr)



messageGraph.edges(data=True,keys=True)

for itr in messageGraph.edges_iter(None, True, True):
    realMessage.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"m", Label= "Message")
    
for itr in tradeGraph.edges_iter(None, True, True):
    realTrade.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"t", Label= "Trade")

for itr in attackGraph.edges_iter(None, True, True):
    realAttack.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"a", Label= "Attack")
    

allianceGraph = nx.compose(realMessage, realTrade)
totalGraph = nx.compose(allianceGraph, realAttack)
print("MESSAGE GRAPH: ", nx.info(messageGraph))
print("")
print("TRADE GRAPH: ", nx.info(tradeGraph))
print("")
print("ATTACK GRAPH: ", nx.info(attackGraph))
print("")
print("ALLIANCE GRAPH: ", nx.info(allianceGraph))
print("")
print("TOTAL GRAPH: ", nx.info(totalGraph))

# print(allianceGraph.degree())
print("")
print("")
# print(totalGraph.degree())

nx.write_graphml(realMessage, baseURL + "Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-30.graphml")

messageGraph = ig.Graph()
messageGraph = messageGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-30.graphml") 

print(messageGraph.summary())

# for edge in messageGraph.es:
#    print(edge)
    
#for vertex in messageGraph.vs:
#    print(vertex)

cont = 0
#print(allNetwork[0][4].indegree())
zeroIndegreeVertex = list()

for vertex in allNetwork[0][4].vs:
    if vertex.indegree() == 0:
        zeroIndegreeVertex.append(vertex)

# print(zeroIndegreeVertex)
suspected = dict()
key = ""
nEdges = 0
nTrades = 0

print("{:>7}{:>7}".format("From", "To"))
for vertex in zeroIndegreeVertex:
    nEdges = 0
    nTrades = 0
    for edge in allNetwork[0][4].es:
        if allNetwork[0][4].vs[edge.tuple[0]] == vertex:
            if edge['Label'] == 'Trade':
                nTrades += 1
            nEdges += 1
    if nEdges == nTrades:
        key = "from:" + allNetwork[0][4].vs[edge.tuple[0]]['id'] +" to:"+ allNetwork[0][4].vs[edge.tuple[1]]['id']
        if key not in suspected:
            suspected[key] = 1
        else:
            suspected[key] += 1

print(suspected)




nGraph = 30
realAllianceGraph = nx.DiGraph()
allianceGraph = nx.MultiDiGraph()

tryGraph = ig.Graph()
tryGraph = tryGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-1.graphml")

edgesDict = dict()

tryCont = 0


tradeCount = 0
messageCount = 0


for i in range(0, 1):
    
    allianceGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-"+str(i+1)+".graphml")
    
    print("START GRAPH")
    print(nx.info(allianceGraph))
    print("")
    for itr in allianceGraph.edges_iter(None, True, True):
        if itr[0] in edgesDict:
                
            if itr[1] in edgesDict[itr[0]]:
                if itr[3]['Label'] == 'Message':
                    edgesDict[itr[0]][itr[1]][0] += 1
                    tryCont += 1
                    messageCount += 1
                elif itr[3]['Label'] == 'Trade':
                    edgesDict[itr[0]][itr[1]][1] += 1
                    tryCont += 1
                    tradeCount += 1
            else:
                if itr[3]['Label'] == 'Message':
                    edgesDict[itr[0]][itr[1]] = [1, 0]
                    tryCont += 1
                    messageCount += 1
                elif itr[3]['Label'] == 'Trade':
                    edgesDict[itr[0]][itr[1]] = [0, 1]
                    tryCont += 1
                    tradeCount += 1
                    
        else:
            edgesDict[itr[0]] = {}
            if itr[3]['Label'] == 'Message':
                edgesDict[itr[0]][itr[1]] = [1, 0]
                tryCont += 1
                messageCount += 1
            elif itr[3]['Label'] == 'Trade':
                edgesDict[itr[0]][itr[1]] = [0, 1]
                tryCont += 1
                tradeCount += 1
    suma = 0
    nStep = 0
    for fromVertex in edgesDict:
        for toVertex in edgesDict[fromVertex]:
            if  realAllianceGraph.has_edge(int(fromVertex), int(toVertex)):
                print("THIS IS A PROBLEM!")
            nStep += 1
            realAllianceGraph.add_edge(int(fromVertex), int(toVertex), Weight = sum(edgesDict[fromVertex][toVertex]))
            '''print("FROM VERTEX: ", fromVertex)
            print("VALUE: ", edgesDict[fromVertex])
            print("TO VERTEX: ", toVertex)
            print("VALUE: ", edgesDict[fromVertex][toVertex])
            print("SUM: ", sum(edgesDict[fromVertex][toVertex]))'''
            suma += sum(edgesDict[fromVertex][toVertex])
            # print("")
            
            #if itr[3]['Label'] = 'Message'
            #edgesDict[itr[0]][itr[1]][0] 
        
            
        # print(itr)

        
# for key, value in edgesDict:
    # print(key)
    # print("")

        
print("VERTEX NUMBER: ", len(edgesDict))
print("ALLIANCE GRAPH LENGHT: ", len(allianceGraph))



cont = 0
for vertex in tryGraph.vs:
    if vertex.outdegree() == 0:
        cont += 1
print("OUTDEGREE = 0 COUNT", cont)
print("")

print("END GRAPH")
print(nx.info(realAllianceGraph))
print("")
cont = 0
for itr in realAllianceGraph.edges_iter(None, True, True):
    cont += itr[2]['Weight']
    
    # print(itr)
print("OUR COUNT: ", cont)
print("CREATION COUNT: ", tryCont)
print("TRADE COUNT: ", tradeCount)
print("MESSAGE COUNT: ", messageCount)
print("TRADE COUNT + MESSAGE COUNT: ", (tradeCount+messageCount))
print("SUMA: ", suma)
print("NUMBER OF STEP: ", nStep)
print('-------------------------------')

    
